In [1]:
import os
import pandas as pd
import numpy as np

C:\tools\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
drivers = pd.read_csv(os.path.join("data_raw/ergast", "drivers.csv"))
results = pd.read_csv(os.path.join("data_raw/ergast", "results.csv"))
races = pd.read_csv(os.path.join("data_raw/ergast", "races.csv"))
circuits = pd.read_csv(os.path.join("data_raw/ergast", "circuits.csv"))
status = pd.read_csv(os.path.join("data_raw/ergast", "status.csv"))
constructors = pd.read_csv(os.path.join("data_raw/ergast", "constructors.csv"))

In [3]:
drivers["driverUrl"] = drivers["url"]

In [4]:
drivers[drivers['surname'].str.contains('Gibson')]

,driverId,driverRef,number,code,forename,surname,dob,nationality,url,driverUrl
601,601,gibson,\N,\N,Dick,Gibson,1918-04-16,British,http://en.wikipedia.org/wiki/Dick_Gibson,http://en.wikipedia.org/wiki/Dick_Gibson


In [5]:
correct_urls = {
    'Ernie_McCoy': 'Ernie_McCoy_(racing_driver)',
    'Jim_Rathmann': 'Jim_Rathmann_(racing_driver)',
    'Tony_Crook':'Tony_Crook_(racing_driver)',
    'David_Piper':'David_Piper_(racing_driver)',
    'Brian_Naylor': 'Brian_Naylor_(racing_driver)',
    'Brian_McGuire':'Brian_McGuire_(racing_driver)',
    'George_Eaton':'George_Eaton_(racing_driver)',
    'Alberto_Colombo':'Alberto_Colombo_(racing_driver)',
    'Julian_Bailey':'Julian_Bailey_(racing_driver)',
    'Geoff_Lees':'Geoff_Lees_(racing_driver)',
    'George_Constantine':'George_Constantine_(racing_driver)',
    'Jean_Lucas':'Jean_Lucas_(racing_driver)',
    'Fran%C3%A7ois_Picard':'Fran%C3%A7ois_Picard_(racing_driver)',
    'Trevor_Taylor':'Trevor_Taylor_(racing_driver)',
    'Arthur_Owen':'Arthur_Owen_(racing_driver)',
    'Tony_Brooks':'Tony_Brooks_(racing_driver)',
    'Dick_Gibson':'Dick_Gibson_(racing_driver)'
}

In [6]:
for c, u in correct_urls.items():
    drivers['driverUrl'] = drivers['driverUrl'].apply(lambda x: x.replace(c, u))

In [7]:
drivers[drivers['surname'].str.contains('Gibson')]

,driverId,driverRef,number,code,forename,surname,dob,nationality,url,driverUrl
601,601,gibson,\N,\N,Dick,Gibson,1918-04-16,British,http://en.wikipedia.org/wiki/Dick_Gibson,http://en.wikipedia.org/wiki/Dick_Gibson_(raci...


In [8]:
drivers = drivers.drop(columns=['url'])

In [9]:
drivers.to_json(os.path.join('data','ergast-drivers.json'), date_format='iso')

In [10]:
results = results.merge(drivers[['driverId', 'driverRef', 'code', 'forename', 'surname', 'dob', 'nationality', 'driverUrl']], on="driverId", how="outer")
circuits["circuitUrl"] = circuits["url"]
circuits["circuit"] = circuits["name"]
races["gp"] = races["name"]
races = races.merge(circuits[["circuitId", "circuitRef", "location", "country", "circuitUrl", "circuit"]], on="circuitId", how="outer")
results = results.merge(races[["raceId","year","round","date","quali_date","quali_time","location","country","gp"]], on="raceId", how="outer")
results = results.merge(status, on="statusId", how="outer")
constructors["constructor"] = constructors["name"]
constructors["constructorUrl"] = constructors["url"]
constructors["constructorNationality"] = constructors["nationality"]
results = results.merge(constructors[["constructorId", "constructor", "constructorNationality", "constructorRef"]], on="constructorId", how="outer")

In [11]:
results["name"] = results["forename"] + " " + results["surname"]
results["date"] = pd.to_datetime(results["date"], format="%Y-%m-%d")
results["quali_date"] = pd.to_datetime(results["date"], format="%Y-%m-%d")
results["dob"] = pd.to_datetime(results["dob"], format="%Y-%m-%d", errors = "ignore")
results["number"] = results["number"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
# results["age"] = results["date"] - results["dob"]
results["year"] = results["year"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
results["round"] = results["round"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
results["position"] = results["position"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
results["fastestLapSpeed"] = results["fastestLapSpeed"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")

C:\Users\micha\AppData\Local\Temp\ipykernel_18592\438737568.py:4: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  results["dob"] = pd.to_datetime(results["dob"], format="%Y-%m-%d", errors = "ignore")


In [12]:
results.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'driverRef', 'code', 'forename',
       'surname', 'dob', 'nationality', 'driverUrl', 'year', 'round', 'date',
       'quali_date', 'quali_time', 'location', 'country', 'gp', 'status',
       'constructor', 'constructorNationality', 'constructorRef', 'name'],
      dtype='object')

In [13]:
results.sort_values(by=['year','round']).dropna().tail(1)['location']

26320    Jeddah
Name: location, dtype: object

In [14]:
results.to_json(os.path.join('data','ergast-results.json'), date_format='iso')

In [15]:
sprints = pd.read_csv(os.path.join("data_raw/ergast", "sprint_results.csv"))

In [16]:
sprints = sprints.merge(drivers[['driverId', 'driverRef', 'code', 'forename', 'surname', 'dob', 'nationality', 'driverUrl']], on="driverId", how="inner")

In [17]:
sprints

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,fastestLap,fastestLapTime,statusId,driverRef,code,forename,surname,dob,nationality,driverUrl
0,1,1061,830,9,33,2,1,1,1,3,...,14,1:30.013,1,max_verstappen,VER,Max,Verstappen,1997-09-30,Dutch,http://en.wikipedia.org/wiki/Max_Verstappen
1,2,1061,1,131,44,1,2,2,2,2,...,17,1:29.937,1,hamilton,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
2,3,1061,822,131,77,3,3,3,3,1,...,17,1:29.958,1,bottas,BOT,Valtteri,Bottas,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas
3,4,1061,844,6,16,4,4,4,4,0,...,16,1:30.163,1,leclerc,LEC,Charles,Leclerc,1997-10-16,Monegasque,http://en.wikipedia.org/wiki/Charles_Leclerc
4,5,1061,846,1,4,6,5,5,5,0,...,16,1:30.566,1,norris,NOR,Lando,Norris,1999-11-13,British,http://en.wikipedia.org/wiki/Lando_Norris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,236,1118,825,210,20,11,16,16,16,0,...,3,1:15.464,1,kevin_magnussen,MAG,Kevin,Magnussen,1992-10-05,Danish,http://en.wikipedia.org/wiki/Kevin_Magnussen
236,237,1118,855,51,24,18,17,17,17,0,...,3,1:15.837,1,zhou,ZHO,Guanyu,Zhou,1999-05-30,Chinese,http://en.wikipedia.org/wiki/Zhou_Guanyu
237,238,1118,807,210,27,12,18,18,18,0,...,3,1:15.472,1,hulkenberg,HUL,Nico,Hülkenberg,1987-08-19,German,http://en.wikipedia.org/wiki/Nico_H%C3%BClkenberg
238,239,1118,822,51,77,14,19,19,19,0,...,3,1:15.738,1,bottas,BOT,Valtteri,Bottas,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas


In [18]:
sprints = sprints.merge(races[["raceId","year","round","location","country","gp"]], on="raceId", how="outer")

In [19]:
sprints["name"] = sprints["forename"] + " " + sprints["surname"]
sprints["year"] = sprints["year"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
sprints["round"] = sprints["round"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
sprints["position"] = sprints["position"].apply(pd.to_numeric, errors = "coerce").astype(np.int64, errors="ignore")
sprints = sprints.merge(constructors[["constructorId", "constructor", "constructorNationality", "constructorRef"]], on="constructorId", how="outer")

In [20]:
sprints

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,driverUrl,year,round,location,country,gp,name,constructor,constructorNationality,constructorRef
0,5.0,1061.0,846.0,1.0,4.0,6.0,5.0,5,5.0,0.0,...,http://en.wikipedia.org/wiki/Lando_Norris,2021.0,10.0,Silverstone,UK,British Grand Prix,Lando Norris,McLaren,British,mclaren
1,6.0,1061.0,817.0,1.0,3.0,7.0,6.0,6,6.0,0.0,...,http://en.wikipedia.org/wiki/Daniel_Ricciardo,2021.0,10.0,Silverstone,UK,British Grand Prix,Daniel Ricciardo,McLaren,British,mclaren
2,23.0,1065.0,817.0,1.0,3.0,5.0,3.0,3,3.0,1.0,...,http://en.wikipedia.org/wiki/Daniel_Ricciardo,2021.0,14.0,Monza,Italy,Italian Grand Prix,Daniel Ricciardo,McLaren,British,mclaren
3,24.0,1065.0,846.0,1.0,4.0,4.0,4.0,4,4.0,0.0,...,http://en.wikipedia.org/wiki/Lando_Norris,2021.0,14.0,Monza,Italy,Italian Grand Prix,Lando Norris,McLaren,British,mclaren
4,46.0,1071.0,846.0,1.0,4.0,7.0,6.0,6,6.0,0.0,...,http://en.wikipedia.org/wiki/Lando_Norris,2021.0,19.0,São Paulo,Brazil,São Paulo Grand Prix,Lando Norris,McLaren,British,mclaren
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550,NaN,1140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2024.0,20.0,Mexico City,Mexico,Mexico City Grand Prix,NaN,NaN,NaN,NaN
1551,NaN,1141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2024.0,21.0,São Paulo,Brazil,São Paulo Grand Prix,NaN,NaN,NaN,NaN
1552,NaN,1142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2024.0,22.0,Las Vegas,United States,Las Vegas Grand Prix,NaN,NaN,NaN,NaN
1553,NaN,1143.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2024.0,23.0,Al Daayen,Qatar,Qatar Grand Prix,NaN,NaN,NaN,NaN


In [21]:
sprints.to_json(os.path.join('data','ergast-sprints.json'), date_format='iso')